# Time Series analysis and Forecasting


In [2]:
# Step 0: Import libraries and load data

In [7]:
# Preprocess of Z_t (serie temporal)

In [ ]:
# Graphic representation of the Z_t

# Exploratory Data Analysis (EDA)
Estochastic process...
Analyzing trend, variance, seasonality and random components.
¿Is variance constant?
¿Is mean constant?
¿Seasonality is present?

In [5]:
# Original Time series decomposition

The results highlights a varying variance in the residuals, with lower variance from 2015–2020 and higher, more volatile variance afterward. This pattern suggests heteroscedasticity, which may impact the performance of an ARIMA model if left unaddressed. One common solution is to apply a log transformation (or sometimes a Box-Cox transformation) to stabilize the variance across the series. This transformation can help reduce the impact of extreme changes and make the series more suitable for ARIMA.

In [9]:
# Log transformation to stabilize the variance

In [ ]:
# Stationarity test (original and differenciated serie)

In [12]:
# Transformed and differenciate time serie decomposition to re-evaluate the stationarity tests and variance to check if this has helped stabilize the residuals.

The ADF and KPSS test results indicate that the original series is non-stationary (with high p-values for ADF and a low p-value for KPSS), but after first differencing, the series becomes stationary (ADF p-value is extremely low, and KPSS p-value is high). This suggests that a differencing order of 1 is appropriate for the ARIMA model.

For the seasonal component, there is annual seasonality based on observation. Since the data is monthly, a seasonal period of 12 is reasonable. This leads to a SARIMA model (Seasonal ARIMA), where the seasonal parameters will capture the year-over-year effects.

# Autocorrelation analysis to etermine Parameters for SARIMA(p,d,q)(P,D,Q,s). 

In [ ]:
# ACF

In [15]:
# PACF

Based on ACF and PACF observations, the parameters for both the non-seasonal and seasonal parts of your SARIMA model are:

Non-Seasonal (p, d, q) Parameters:
Since the ACF and PACF both show a significant spike at lag 1 and then drop to zero, this suggests a simple structure for the non-seasonal AR and MA components:

AR (p) = 1: The spike in the PACF at lag 1 suggests a single autoregressive term.
MA (q) = 1: The spike in the ACF at lag 1 also suggests a single moving average term.
Differencing (d) = 1: Based on the stationarity test results, established before, a first difference (d = 1) is appropriate to make the series stationary.

Thus, for the non-seasonal component, ARIMA (p, d, q) = (1, 1, 1).

Seasonal (P, D, Q, s) Parameters:
With a monthly frequency and clear annual seasonality, the seasonal period s is 12.

From the PACF and ACF patterns following seasonal parameters are determined:

Seasonal AR (P) = 1: The significant spike at lag 12 in the PACF indicates that a seasonal autoregressive component of order 1 could be suitable.
Seasonal Differencing (D) = 1: Given the annual seasonality, D = 1 is typical to remove seasonal trends.
Seasonal MA (Q) = 1: Since the ACF shows a spike at lag 12, but below the confidence interval, Q=0 is considered.
For the seasonal part (P, D, Q, s) = (1, 1, 0, 12).

Based on these parameters, the SARIMA model would be:

**SARIMA(1, 1, 1)(1, 1, 1, 12)**

# SARIMA Model fitting 

In [20]:
# Split Data in Train and Test

In [22]:
# Fit SARIMA(1,1,1)(1,1,0,12) model

In [24]:
# Diagnostic

Interpret Diagnostic

In [ ]:
# Predictions

# Model tuning
Optimizing hyperparameters (p,d,q)(P,D,Q) using grid-search techniques and auto-arima.

In [28]:
# Step 1: Split in train validation and test

# Plot train, validation and test

## Grid Search basecd on backtesting

In [ ]:
# Grid search based on backtesting

## Auto-Arima

In [32]:
# Auto arima: selection based on AIC

## Predictions

In [35]:
# Backtest predictions with the original model SARIMA(1,1,1)(1,1,0,12)

# Backtest predictions with the best model according to grid search

# Backtest predictions with the best model according to auto-arima


In [37]:
# Compare predictions (MAE)

# Plot Actual vs Predicted

# Conclusion and following steps